In [ ]:
# ASSNP-09.py
# 2018-11-29
#
# 從 dem-all-7.csv文字檔 載入資料
# delimiter=',' --> 定界符是','
# unpack=True   --> transposed array (將陣列進行轉置)
# x,y,z = np.loadtxt(path_data + 'dem_all-7.csv', delimiter=',', unpack=True)
# 將轉置後的陣列行解包到指定的變數x,y,z中。
#
from datetime import datetime
dt_start = datetime.now()
#
import os
# to fix matplotlib warning
os.environ['MPLCONFIGDIR'] = os.getcwd() + "/configs/"
path = os.getcwd()
print('當前工作目錄 ==>', path)
#
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fontManager
#
path_data    = './data/'
path_graphic = './graphic/'
#
matplotlib.rcParams['pdf.fonttype'] = 42
myfont = fontManager.FontProperties(fname='/home/jovyan/fonts/msjh.ttc')
# ----------------------------------------------------------------------
#
# Left Down point: 0, 2040
# Right Up point:  3800, 9000
# height (min, max) 0, 535.81
#
# 3D graphic x,y factor = 0.01
# 3D graphic z   factor = 0.01 * 1.5
#
fig = plt.figure(figsize=(70, 70))
ax  = fig.add_subplot(1,1,1, projection='3d')
#
plt.xlim( 0, 70)    # xlim, ylim, zlim need to be same
plt.ylim(20, 90)    
ax.set_zlim(0, 50)
#
x_ticks = np.linspace( 0,70,8)
y_ticks = np.linspace(20,90,8)
z_ticks = np.linspace( 0,50,6)
#
plt.xticks(x_ticks)
plt.yticks(y_ticks)
plt.tick_params(labelsize=40)
#
plt.xlabel   ('X座標', fontproperties=myfont, fontsize=46)
plt.ylabel   ('Y座標', fontproperties=myfont, fontsize=46)
ax.set_zlabel('Z座標', fontproperties=myfont, fontsize=46)
#
ax.xaxis.label.set_color('red')
ax.yaxis.label.set_color('red')
ax.zaxis.label.set_color('red')
#
# 此程式碼使輸出之3D圖像Z軸比例更正確
plt.gca().set_aspect('auto', adjustable='box') # set X,Y same ratio & scale
#
# ---------------------------------------------------------
#
# 逐筆讀入資料 dem-all-7.csv
# Read from the file into the array data(:,:)
# array data shape is (...., 3)
#
# Columns.
# data[..., 0] is array of float number, 二度分帶 X座標
# data[..., 1] is array of float number, 二度分帶 Y座標
# data[..., 2] is array of float number, DEM file z value
#
# 台灣橫麥卡托二度分帶投影座標系統（TM2）
#
x,y,TM2_Z = np.loadtxt(path_data + 'dem-all-7.csv', delimiter=',', unpack=True)
# XYlist is sorted (x, y), first order is y, then x
XYlist = list(zip(x,y))
#
print('Left Down point:   ', int(np.amin(x)), ',', int(np.amin(y)))
print('Right Up  point:',    int(np.amax(x)), ',', int(np.amax(y)))
print('height(min, max)',     np.amin(TM2_Z), ',', np.amax(TM2_Z))
#
# Left Down point:    0 , 2040
# Right Up  point: 3800 , 9000
# height(min, max)  0.0 , 357.21
#
# X, Y value
X = np.arange(-100, 3820, 20)
Y = np.arange(2040, 9020, 20)
#
# X -100 .. 3800
# Y 2040 .. 9000
# 6980/20 ==> 349
# 3920/20 ==> 196
#
Xmesh, Ymesh = np.meshgrid(X, Y)  # x-y 平面的網格
Zmesh = np.zeros(shape=(349,196),dtype=np.float16)
#
i = 0 ; j = 0
for y in np.arange(2040, 9020, 20) :
    print('y is', y)
    z = np.zeros(shape=(196),dtype=np.float16)
    zi = 0
    for x in np.arange(-100, 3820, 20):
        if (x,y) in XYlist :
            z[zi] = TM2_Z[i]
            i = i + 1
        else :
            z[zi] = 0.0
        # end if
        zi = zi + 1
    # end for
    Zmesh[j] = z
    j = j + 1
#end for
#
print ('\n')
#
factor_3D = 0.01
Xmesh_3D = factor_3D * Xmesh
Ymesh_3D = factor_3D * Ymesh
Zmesh_3D = factor_3D * Zmesh * 1.5
#
# cstride	Array column stride (step size)
# rstride	Array row    stride (step size)
#
ax.plot_surface(Xmesh_3D, Ymesh_3D, Zmesh_3D, cstride=1, rstride=1, cmap=plt.cm.terrain)
ax.view_init(elev=45, azim=230)
#
# 設定子圖的標題
str_title = 'ASSNP-09 台灣橫麥卡托二度分帶投影座標系統（TM2）'
plt.title(str_title, fontproperties=myfont, fontsize=60)
plt.draw()
#
# bbox_inches='tight' is for display ylabel
# on myBinder.org jupyterLab platform, set dpi=300 will make kernel die
#
plt.savefig(path_graphic + "ASSNP-09.png", format='png', dpi=150, bbox_inches='tight')
# plt.savefig(path_graphic + "ASSNP-09.pdf", dpi=300, bbox_inches='tight')
#
print(dt_start)
dt_end = datetime.now()
print(dt_end)
#
print('done')
#